In [1]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eren\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eren\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eren\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
df= pd.read_csv("stemmed_sentences.csv")

In [7]:
df

,major small larg bowel procedur with complic comorbid percutan cardiovascular procedur without acut myocardi infarct without coronari arteri stent implant dental oral di except extract restor age craniotomi age except for trauma disord of liver except malign cirrhosi alcohol hepat with complic comorbid seizur headach age with complic comorbid cardiac arrhythmia conduct disord with complic comorbid extracrani vascular procedur perman cardiac pacemak implant with acut myocardi infarct heart failur or shock or aicd lead or gener proc septicemia age simpl pneumonia pleurisi age with complic comorbid coronari bypass without cardiac cathet lower extrem humeru procedur except hip foot femur age complic comorbid circulatori disord with acut myocardi infarct major complic discharg aliv nervou system neoplasm with complic comorbid nervou system infect except viral mening kidney urinari tract infect age with complic comorbid septicemia age respiratori infect inflamm age with complic comorbid extens oper room procedur unrel to princip diagnosi septicemia age respiratori system diagnosi with ventil support percutan cardiovascular procedur with stent without acut myocardi infarct circulatori disord with acut myocardi infarct major complic discharg aliv septicemia age septicemia age liver transplant ecmo or tracheostomi with mechan ventil hour or princip diagnos except face mouth and neck diagnos with major oper room procedur septicemia age lymphoma leukemia with complic comorbid respiratori system diagnosi with ventil support septicemia age craniotomi for trauma craniotomi age with complic comorbid peptic ulcer gastriti gastrointestin hemorrhag with complic comorbid cholecystectomi with common duct explor with complic comorbid other oper room procedur for multipl signific trauma intracrani hemorrhag or cerebr infarct extens procedur unrel to princip diagnosi extens oper room procedur unrel to princip diagnosi septicemia dissemin infect septicemia age other endocrin disord endocrin disord with complic comorbid shoulder upper arm forearm procedur major joint limb reattach procedur of upper extrem cardiac valv procedur cardiac catheter cardiac valv other major cardiothorac proc without cardiac cathet ecmo or tracheostomi with mechan ventil hour or princip diagnos except face mouth and neck diagnos with major oper room procedur musculoskelet other procedur for multipl signific trauma other endocrin disord endocrin disord with complic comorbid connect tissu disord connect tissu disord with complic comorbid infecti parasit diseas includ hiv w
0,procedur infecti parasit diseas w or procedur ...
1,procedur infecti parasit diseas w or procedur ...
2,procedur heart failur heart failur heart failu...
3,tracheostomi w long term mechan ventil w exten...
4,obstruct w mcc other pneumonia other pneumonia...
5,procedur infecti parasit diseas w
6,procedur w mcc infecti parasit diseas includ h...
7,procedur digest malign digest malign other dig...
8,procedur w mcc septicemia dissemin infect sept...
9,procedur for neoplasm other


In [9]:
df.columns = ["0"]

# NaN değerleri ve boş stringleri temizle
df = df.dropna()
df = df[df["0"].str.strip() != ""]

# Temizlenmiş ilk 5 cümleyi göster
print(df.head())

                                                   0
0  procedur infecti parasit diseas w or procedur ...
1  procedur infecti parasit diseas w or procedur ...
2  procedur heart failur heart failur heart failu...
3  tracheostomi w long term mechan ventil w exten...
4  obstruct w mcc other pneumonia other pneumonia...


In [11]:
#Varsayalım ki DataFrame'iniz df ve yorumlar '0' sütununda
text = ' '.join(df['0'].astype(str).tolist())  # Tüm yorumları tek bir metin haline getir
sentences = sent_tokenize(text)  # Metni cümlelere ay

In [15]:
stop_words = set(stopwords.words('english'))

In [17]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize

# Lemmatizer ve Stemmer nesnelerini oluştur
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
# Kelimeleri tokenleştirip, lemmatize etme ve stemleme
def preprocess_sentence(sentence):
    tokens = word_tokenize(sentence)
    filtered_tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    return lemmatized_tokens, stemmed_tokens

# Kullanım örneği
tokenized_corpus_lemmatized = []
tokenized_corpus_stemmed = []

for sentence in sentences:
    lemmatized_tokens, stemmed_tokens = preprocess_sentence(sentence)
    tokenized_corpus_lemmatized.append(lemmatized_tokens)
    tokenized_corpus_stemmed.append(stemmed_tokens)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
# Ön işlenmiş token listelerini tekrar metne çeviriyoruz
lemmatized_texts = [' '.join(tokens) for tokens in tokenized_corpus_lemmatized]

lemmatized_texts[:3]

['procedur infecti parasit diseas w procedur cirrhosi alcohol hepat disord liver disord liver cirrhosi alcohol hepat major stomach esophag duoden procedur stomach esophag duoden proc age w cc major gi dx infecti parasit diseas includ hiv w procedur infecti parasit diseas w procedur malfunct reaction comp orthoped devic procedur aftercar musculoskelet system connect tissu hip femur procedur trauma except joint replac hip femur procedur except major joint age complic comorbid respiratori malign respiratori neoplasm circulatori disord acut myocardi infarct expir acut myocardi infarct poison toxic effect drug age complic comorbid poison medicin agent intracrani hemorrhag intracrani hemorrhag cerebr infarct respiratori system diagnosi ventil support hour respiratori system diagnosi ventil support hr multipl signific trauma multipl signific trauma procedur heart failur heart failur heart failur shock ecmo trach w mv hr pdx exc face mouth neck w maj tracheostomi w long term mechan ventil w ex

In [23]:
# TF-IDF vektörizerı başlatıyoruz
vectorizer = TfidfVectorizer()

# TF-IDF matrisini oluşturuyoruz
#terim frekansları, belge frekanslarıni hesplar
#TF-IDF vektörlerine dönüştürür
tfidf_matrix = vectorizer.fit_transform(lemmatized_texts)

## Kelimeleri alalım
#F-IDF vektörleştirme işleminde kullanılan tüm kelimelerin essiz bir listesini döndürur
feature_names = vectorizer.get_feature_names_out()

# TF-IDF matrisini pandas DataFrame'e çevir-gorunurluk acisindan- calismasi kolay
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# İlk birkaç satırı gösterelim-ilk 5 cümle
print(tfidf_df.head())

#Her satır bir cümleyi temsil eder
#Her sütun bir kelimeyi temsil eder
#Hücreler ise o kelimenin o cümledeki TF-IDF skorudur - her cumle icin degisir-bakiniz:slaytlar

       acut  aftercar       age     agent       alc   alcohol      aliv  \
0  0.090233  0.007519  0.022558  0.007519  0.015039  0.022558  0.007519   

        ami  arrhythmia      back  ...  tracheostomi     tract    trauma  \
0  0.037597    0.022558  0.022558  ...      0.045117  0.112792  0.075194   

    traumat     unrel      uret   urinari  vascular    ventil     wound  
0  0.015039  0.007519  0.007519  0.067675  0.007519  0.150389  0.015039  

[1 rows x 165 columns]


In [25]:
tfidf_df.to_csv('tfidf_stemmed.csv', index=False)